In [1]:
from __future__ import absolute_import
import aiida
from aiida import load_profile
load_profile()

# Import commonly used functionality
import numpy as np
from aiida import orm, engine, common
from aiida.plugins import WorkflowFactory
from aiida.orm import Code


In [2]:
!reentry scan
!verdi daemon restart
#!verdi daemon start

Restarting the daemon... OK


In [3]:
from aiida_defects.formation_energy_siesta.formation_energy_siesta import FormationEnergyWorkchainSIESTA
#import .formation_energy_siesta
#!pwd

In [4]:
from aiida.orm import StructureData
#Structure Pure
cell = [[15.0, 0.0, 0.0,],
        [ 0.0,15.0, 0.0,],
        [ 0.0, 0.0,15.0,],
        ]
pure = StructureData(cell=cell)
pure.append_atom(position=( 0.000 , 0.000 , 0.000 ),symbols=['O']) #1
#pure.append_atom(position=( 0.757 , 0.586 , 0.000 ),symbols=['H']) #2
#pure.append_atom(position=(-0.757 , 0.586 , 0.000),symbols=['H']) #3 
#pure.append_atom(position=( 0.000 , 3.500 , 0.000),symbols=['O']) #4
#pure.append_atom(position=( 0.757 , 2.914 , 0.000 ),symbols=['H']) #5
#pure.append_atom(position=(-0.757 , 2.914 , 0.000),symbols=['H']) #6


defect=StructureData(cell=cell)
defect.append_atom(position=( 0.000 , 0.000 , 0.000 ),symbols=['O']) #1
#defect.append_atom(position=( 0.757 , 0.586 , 0.000 ),symbols=['H']) #2
#defect.append_atom(position=(-0.757 , 0.586 , 0.000),symbols=['H']) #3 
#defect.append_atom(position=( 0.000 , 3.500 , 0.000),symbols=['O']) #4
#defect.append_atom(position=( 0.757 , 2.914 , 0.000 ),symbols=['H']) #5
#defect.append_atom(position=(-0.757 , 2.914 , 0.000),symbols=['H'],name="GhostH") #6

In [5]:
#code = Code.get_from_string('siesta-psml-lua@N552VW')
code = Code.get_from_string('siesta-master-gfortran-openmpi-lua@icn2-workstation')

charge=-2

In [6]:
from aiida.orm import Dict
parameters_host = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "cg",
    "MD-NumCGsteps": "5000",
   #"LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang", 
    })
parameters_defect_q0 = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "cg",
    "MD-NumCGsteps": "5000",
   #"LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang", 
   "NetCharge": "0",
    })
parameters_defect_q = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "cg",
    "MD-NumCGsteps": "5000",
   #"LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang",
   "NetCharge": str(charge),
    })
#options_host=Dict(dict={'options':{"max_wallclock_seconds": 360},
#                        "resources":{"num_machines": 1,
#                                      "num_mpiprocs_per_machine": 1}
#                        })
options_host=Dict(dict={
        "max_wallclock_seconds": 360000,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    })
options_defect_q0=Dict(dict={
        "max_wallclock_seconds": 360000,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    })
options_defect_q=Dict(dict={
        "max_wallclock_seconds": 360000,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    })



In [7]:
basis_dict_host =Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
O DZP
#H    DZP
%endblock pao-basis-sizes""",
})
basis_dict_defect_q0 = Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
O DZP
#GhostH DZP
#H    DZP
%endblock pao-basis-sizes""",
})
basis_dict_defect_q =Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
O DZP
#GhostH DZP
#H    DZP
%endblock pao-basis-sizes""",
})

In [8]:
parameters_defect_q0.get_dict()

{'DM-History-Depth': '0',
 'DM-NumberPulay ': '3',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'MD-NumCGsteps': '5000',
 'MD-TypeOfRun': 'cg',
 'MD-VariableCell': 'F',
 'NetCharge': '0',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'SCF-Mixer-weight': '0.02',
 'dm-tolerance': '0.0001',
 'mesh-cutoff': '250 Ry'}

In [9]:
kpoints_host = orm.KpointsData()
kpoints_host.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly
kpoints_defect_q0 = orm.KpointsData()
kpoints_defect_q0.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly
kpoints_defect_q = orm.KpointsData()
kpoints_defect_q.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly

In [10]:
import os
from aiida_siesta.data.psf import PsfData
pseudo_file_to_species_map = [ ("O.psf", ['O'])]
#pseudo_file_to_species_map = [ ("H.psf", ['H']),("O.psf", ['O'])]
#pseudo_file_to_species_map = [ ("H.psf",["H"]),("GhostH.psf",["GhostH"]),("O.psf", ['O'])]
pseudos_dict_host = {}
for fname, kinds, in pseudo_file_to_species_map:
      absname = os.path.realpath(os.path.join("./pseudos",fname))
      pseudo, created = PsfData.get_or_create(absname, use_first=True)
      for j in kinds:
              pseudos_dict_host[j]=pseudo                
pseudos_dict_host


{'O': <PsfData: uuid: fa7fad04-23b3-4b53-8f07-ef4dc431be5a (pk: 2)>}

In [11]:
pseudo_file_to_species_map = [("O.psf", ['O'])]
#pseudo_file_to_species_map = [ ("H.psf", ['GhostH','H']),("O.psf", ['O'])]
pseudos_dict_defect = {}
for fname, kinds, in pseudo_file_to_species_map:
      absname = os.path.realpath(os.path.join("./pseudos",fname))
      pseudo, created = PsfData.get_or_create(absname, use_first=True)
      for j in kinds:
              pseudos_dict_defect[j]=pseudo
pseudos_dict_defect

{'O': <PsfData: uuid: fa7fad04-23b3-4b53-8f07-ef4dc431be5a (pk: 2)>}

In [12]:
inputs = {
    # Structures
    'host_structure': pure,
    'defect_structure': defect,
    # Defect information 
    #'defect_charge' : orm.Float(-2.),
    'defect_charge' : orm.Float(charge),
    'defect_site' : orm.List(list=[-0.757 , 2.914 , 0.000]),    # Position of the defect in crystal coordinates
    'fermi_level' : orm.Float(0.0),               # Position of the Fermi level, with respect to the valence band maximum      
    #'chemical_potential' : orm.Float(250.709), # eV, the chemical potentical of a C atom
    #'gaussian_sigma':orm.Float(0.5),
    'correction_scheme' : orm.Str('none'),
    #"epsilon":orm.Float(1.0),
    "pseudos_host":pseudos_dict_host,
    "pseudos_defect":pseudos_dict_defect,
    #"pseudos_q0":pseudos_dict,
    #"pseudos_q":pseudos_dict,
    # Computational (chosen code is QE)
    'siesta' : { 'dft': {'supercell_host':{'code': code, 'kpoints': kpoints_host, 'parameters' : parameters_host,
                                           'options':options_host,"basis": basis_dict_host},
                         'supercell_defect_q0':{'code': code, 'kpoints': kpoints_defect_q,'parameters' : parameters_defect_q0,
                                                'options':options_defect_q0,"basis": basis_dict_defect_q0},
                         'supercell_defect_q':{'code': code, 'kpoints': kpoints_defect_q,'parameters' : parameters_defect_q,
                                               'options':options_defect_q,"basis": basis_dict_defect_q}
}}}

In [13]:
inputs

{'correction_scheme': <Str: uuid: 8e79b124-920c-40cb-b263-748a04e42c61 (unstored) value: none>,
 'defect_charge': <Float: uuid: 4fd6c5f4-62d5-4de9-b0a7-203a45439500 (unstored) value: -2.0>,
 'defect_site': <List: uuid: 8a830286-8b45-4866-a780-17fee9c3fc96 (unstored) value: [-0.757, 2.914, 0.0]>,
 'defect_structure': <StructureData: uuid: 303d382d-4cf9-472b-b6e5-3ae03064a149 (unstored)>,
 'fermi_level': <Float: uuid: ee7e2027-23bb-4766-9673-8792b3ea70a6 (unstored) value: 0.0>,
 'host_structure': <StructureData: uuid: 50583783-7bfb-48a3-b7e3-ddb6e4f25e22 (unstored)>,
 'pseudos_defect': {'O': <PsfData: uuid: fa7fad04-23b3-4b53-8f07-ef4dc431be5a (pk: 2)>},
 'pseudos_host': {'O': <PsfData: uuid: fa7fad04-23b3-4b53-8f07-ef4dc431be5a (pk: 2)>},
 'siesta': {'dft': {'supercell_defect_q': {'basis': <Dict: uuid: 680b86b2-cc27-41be-bfb6-1c4232b6cb94 (unstored)>,
    'code': <Code: Remote code 'siesta-master-gfortran-openmpi-lua' on icn2-workstation, pk: 1, uuid: 7f381566-466a-4216-baa3-21027ddbe15

In [14]:
workchain_future = engine.submit(FormationEnergyWorkchainSIESTA, **inputs)

In [15]:
workchain_future

<WorkChainNode: uuid: 08807c10-14a2-4d49-817d-258e137d0966 (pk: 42) (aiida.workflows:defects.formation_energy_siesta.siesta)>

In [16]:
#!verdi process list -a -p 1

In [17]:
#!verdi process kill 2669

In [18]:
#!verdi process show 3160

In [19]:
#!verdi process report 3160

In [20]:
#workchain_future.outputs.formation_energy_uncorrected.value #.outputs.formation_energy_uncorrected.value
#from aiida.orm import load_node
#results=load_node('2952')

In [21]:
#results.outputs['output_parameters'].get_dict()#['Etot']
#results['forces_and_stress']

In [22]:
#results.outputs.output_parameters.get_dict()

In [23]:
#code

In [24]:
#from aiida_siesta.calculations.siesta import SiestaCalculation
#builder = SiestaCalculation.get_builder()

In [25]:
#builder.metadata